# ATP Synthase Model attempt 6: Include Efflux Pump
In this notebook we will add an efflux pump to model the consistent regeneration of H+ - will not have to have a specific input. Could be compatible with biocrnpyler!

In [1]:
# Import cell (refined)
import numpy as np
import pandas as pd 

import scipy.integrate
from scipy.integrate import odeint

from biocrnpyler import *
from bioscrape import *
from bioscrape.types import Model
from bioscrape.simulator import py_simulate_model



import colorcet

import bokeh.io
import bokeh.plotting
from bokeh.layouts import row

bokeh.io.output_notebook()

Loading BokehJS ...

## Model Membrane Integration, ATP Synthase, and Efflux Pump
### scipy.odeint

----
``` 
DNA_atpsyn --> (k4_tx) RNA_atpsyn
RNA_atpsyn --> (k5_tl) ATP Synthase (protein folding)
ATP Synthase_free --> (k6_mem) ATP Synthase_bound

```
------
``` 
DNA_efflux --> (k4_tx) RNA_efflux
RNA_efflux --> (k5_tl) Efflux Pump (protein folding)
Efflux Pump_free --> (k6_mem) Efflux Pump_bound

```
------
When 1 ATP synthase bound:
```
if ph outside <= 8: (ATP synthesis)
    ADP + Pi + 4 H+ --> (k1) ATP
    ATP --> (k2) ADP + Pi
    outside_H --> (k3) H+
    
elif ph outside > 8: (ATP hydrolysis)
    ATP + 4 H + --> (k1) ADP + Pi
    H+ --> (k3) outside_H
    
```
----
When 1 Efflux pump bound:
```
H+ -->(k7_efflux) outside_H
```

In [2]:
def rhs(var, t, k1, k2, k3, k4_tx, k5_tl, k6_mem, k7_efflux):
    
    atp, adp, pi, h, h20, outside_H, dna_atpsyn, rna_atpsyn, atpsyn_f, atpsyn_b,\
    dna_efflux, rna_efflux, efflux_f, efflux_b= var
    
    # atp synthase
    ddna_atpsyn_dt = -k4_tx * dna_atpsyn
    
    drna_atpsyn_dt = k4_tx * dna_atpsyn - k5_tl * rna_atpsyn
    
    datpsyn_f_dt = k5_tl * rna_atpsyn - k6_mem * atpsyn_f
    
    datpsyn_b_dt = k6_mem * atpsyn_f
    
    # efflux pump
    ddna_efflux_dt = -k4_tx * dna_efflux
    
    drna_efflux_dt = k4_tx * dna_efflux - k5_tl * rna_efflux
    
    defflux_f_dt = k5_tl * rna_efflux - k6_mem * efflux_f
    
    defflux_b_dt = k6_mem * atpsyn_f
    
    
    
    
    # PH CONTROL PORTION WHEN PH IS GREATER THAN 8, ATP SYNTHASE RUNS BACKWARDS (assume has same rxn rate:(
    ph = np.log10(outside_H)
#     if ph > 8 and atpsyn_b > 0.9 and efflux_b > 0.9:
#         print('no')
#         datp_dt = -k1 * atp * h**4
    
#         dadp_dt = k1 * atp * h**4
    
#         dpi_dt = k1 * atp * h**4
    
#         dh_dt = -k3 * h
        
#         dh20_dt = 0

#         doutsideH_dt = k3 * h + k7_efflux * h
        
      
        
    if ph <= 8 and atpsyn_b > 0.9 and efflux_b > 0.9:
       # print('yes')
        
        datp_dt = k1 * adp * pi * h**3.3 - k2 * atp       
        #datp_dt = k1 * adp * pi - k2 * atp

       # dadp_dt = -k1 * adp * pi + k2 * atp
        
        dadp_dt = -k1 * adp * pi * h**3.3 + k2 * atp

        dpi_dt = dadp_dt

        dh_dt = k3 * outside_H - k1 * adp * pi * h**4 - k7_efflux * h
        #dh_dt = k3 * outside_H - k1 * adp * pi  - k7_efflux * h


        dh20_dt = k1 * adp * pi * h**3.3
        #dh20_dt = k1 * adp * pi 

        doutsideH_dt = -k3 * outside_H + k7_efflux * h
        
        
        
    else:
        datp_dt = 0
        
        dadp_dt = 0
        
        dpi_dt = 0
        
        dh_dt = 0
        
        dh20_dt = 0
        
        doutsideH_dt = 0
        
        
    
    return np.array([datp_dt, dadp_dt, dpi_dt, dh_dt, dh20_dt, doutsideH_dt,
                     ddna_atpsyn_dt, drna_atpsyn_dt, datpsyn_f_dt, datpsyn_b_dt,
                    ddna_efflux_dt, drna_efflux_dt, defflux_f_dt, defflux_b_dt])

In [3]:
def rhs_noatpsyn(var,t,k2):
    
    atp,adp,pi = var

#     ddna_dt = -k4 * dna
    
#     drna_dt = k4 * dna - k5 * rna
    
#     datpase_f_dt = k5 * rna - k6 * atpase_f
    
#     datpase_b_dt = k6 * atpase_f
    
    
    datp_dt = -k2 * atp
    
    dadp_dt = k2 * atp
    
    dpi_dt = k2 * atp
    
    return np.array([datp_dt, dadp_dt, dpi_dt])

In [4]:
t_vals = np.linspace(0,100,201)
# k1 = 100 * 60 * 60 # per sec to hr
# k2 = 100 * 60 * 60
# k3 = 333 * 60 * 60 # per sec to hr

k1 = 0.1 # per sec to hr
k2 = 0.2
k3 = 0.1 # per sec to hr
k4_tx = 1
k5_tl = 1
k6_mem = 1
k7_efflux = 1

x_args = (k1, k2, k3, k4_tx, k5_tl, k6_mem, k7_efflux)

atp_0 = 30
adp_0 = 0
pi_0 = 0
h_0 = 0
h20_0 = 0 
outside_H = 30 # 1.5
dna_atpsyn = 1
rna_atpsyn = 0
atpsyn_f = 0
atpsyn_b = 0
dna_efflux = 1
rna_efflux = 0
efflux_f = 0
efflux_b = 0

y0 = np.array([atp_0, adp_0, pi_0, h_0, h20_0, outside_H, dna_atpsyn, rna_atpsyn, atpsyn_f, atpsyn_b,
              dna_efflux, rna_efflux, efflux_f, efflux_b])

x = odeint(rhs, y0, t_vals, args = x_args)


#colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02']
palette = colorcet.b_glasbey_category10


p = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",
    title = 'All Species with ATP Synthase'
        )
p1 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Outside H+'
        )

p2 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'No ATP Synthase Control'
        )


p3 = bokeh.plotting.figure(
        frame_width=450,
        frame_height=200,
        x_axis_label="time (hrs)",
        y_axis_label="concs (mM)",title = 'Membrane Integration Components'
        )


r_atp = x[:,0]
r_adp = x[:,1]
r_pi = x[:,2]
r_h = x[:,3]
r_h20 = x[:,4]
r_outside_H = x[:,5]
r_dna_atpsyn = x[:,6]
r_rna_atpsyn = x[:,7]
r_atpsyn_f = x[:,8]
r_atpsyn_b = x[:,9]
r_dna_efflux = x[:,10]
r_rna_efflux = x[:,11]
r_efflux_f = x[:,12]
r_efflux_b = x[:,13]

plt_list = [r_dna_atpsyn, r_rna_atpsyn, r_atpsyn_f, r_atpsyn_b, r_dna_efflux, r_rna_efflux, r_efflux_f, r_efflux_b]
label = ['dna atpsyn', 'rna atpsyn', 'atpsyn f', 'atpsyn b', 'dna efflux', 'rna efflux', 'efflux f', 'efflux b']
for i,spec in enumerate(plt_list):
    
    p3.line(t_vals, spec, line_width = 2, color = palette[i], legend_label = label[i])
    
p3.legend.click_policy = 'hide'


p.line(t_vals, r_atp, line_width = 2, color = palette[0], legend_label = 'atp')
p.line(t_vals, r_adp, line_width = 2, color = palette[1], legend_label = 'adp')
p.line(t_vals, r_h, line_width = 2, color = palette[2], legend_label = 'h')


p1.line(t_vals, r_outside_H, line_width = 2, color = palette[3], legend_label = 'outside H')
#p1.line(t_vals, r_h20, line_width = 2, color = palette[1], legend_label = 'h20')

noatpsyn_spec_list = ['atp', 'adp', 'pi']
noatpsyn_0 = np.array([atp_0, adp_0, pi_0])
noatpsyn_args = (k2,)

noatp = odeint(rhs_noatpsyn, noatpsyn_0, t_vals, args = noatpsyn_args)

for i,spec in enumerate(noatpsyn_spec_list):
    p2.line(t_vals, noatp[:,i], line_width = 2, color = palette[i], legend_label = spec)



bokeh.io.show(row(p,p1))
bokeh.io.show(row(p2,p3))

Membrane integration components same for atp synthase and efflux pump (as expected based on chosen parameters)

**H+ is not being regenerated - something to do with the 4 requirement I think**

## Model Membrane Integration, ATP Synthase, and Efflux Pump
### bioscrape/biocrnpyler

---
```
if ph outside <= 8: (ATP synthesis)
    ADP + Pi (+ 4 H+) --> (k1) ATP
    ATP --> (k2) ADP + Pi
    outside_H --> (k3) H+
    
```
----
When 1 Efflux pump bound:
```
H+ -->(k7_efflux) outside_H
```

In [5]:
species = ['atp','adp', 'pi', 'h', 'outside_h']
parameters = [('k1', 0.1), ('k2', 2), ('k3', 1), ('k7_efflux', 1)]

rxn1 = (['adp', 'pi'], ['atp'], 'massaction', {'k':'k1'})
rxn1_h = (['adp', 'pi', 'h', 'h', 'h', 'h'], ['atp', 'h', 'h', 'h', 'h'], 'massaction', {'k':'k1'})
rxn2 = (['atp'], ['adp', 'pi'], 'massaction', {'k':'k2'})
rxn3 = (['outside_h'], ['h'], 'massaction', {'k':'k3'})
rxn4 = (['h'], ['outside_h'], 'massaction', {'k':'k7_efflux'})

reactions = [rxn1, rxn2, rxn3, rxn4]

reactions_noatpsyn = [rxn2]

reactions_withH = [rxn1_h, rxn2, rxn3, rxn4]

x_0 = {
    
    'atp':30,
    'outside_h':15
}


M = Model(species = species, reactions = reactions, parameters = parameters,
         initial_condition_dict = x_0)

M_noatpsyn =  Model(species = species, reactions = reactions_noatpsyn, parameters = parameters,
         initial_condition_dict = x_0)

M_withH = Model(species = species, reactions = reactions_withH, parameters = parameters,
         initial_condition_dict = x_0)

timepoints = np.linspace(0,5,1000)
df_results = py_simulate_model(timepoints, Model = M, stochastic = False)
df_results_noatpsyn = py_simulate_model(timepoints, Model = M_noatpsyn, stochastic = False)
df_results_withH = py_simulate_model(timepoints, Model = M_withH, stochastic = False)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, h, 
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, h, 
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: The following species are uninitialized and their value has been defaulted to 0: adp, pi, h, 


In [6]:
palette = colorcet.b_glasbey_category10

p = bokeh.plotting.figure(width = 500, height = 400, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = 'WITH ATP SYNTHASE')

p1 = bokeh.plotting.figure(width = 500, height = 400, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = 'WITHOUT ATP SYNTHASE')

p2 = bokeh.plotting.figure(width = 500, height = 400, 
                             x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration (mM)',
                             title = 'WITH ATP SYNTHASE and H+ (on both ends)')

p.line(timepoints, df_results['atp'], color = palette[0], line_width = 2,legend_label = 'atp')
p.line(timepoints, df_results['adp'], color = palette[1],line_width = 2, legend_label = 'adp')
p.line(timepoints, df_results['outside_h'], color = palette[4], line_width = 2,legend_label = 'outside_h')
p.line(timepoints, df_results['h'], color = palette[3], line_width = 2,legend_label = 'h')

p1.line(timepoints, df_results_noatpsyn['atp'], color = palette[0], line_width = 2,legend_label = 'atp')

p2.line(timepoints, df_results_withH['atp'], color = palette[0], line_width = 2,legend_label = 'atp')
p2.line(timepoints, df_results_withH['adp'], color = palette[1],line_width = 2, legend_label = 'adp')
p2.line(timepoints, df_results_withH['outside_h'], color = palette[4], line_width = 2,legend_label = 'outside_h')
p2.line(timepoints, df_results_withH['h'], color = palette[3], line_width = 2,legend_label = 'h')

bokeh.io.show(row(p,p1))
bokeh.io.show(p2)

- When we simulate with only rxn1 and rxn2, we get a constant of outside_H and h at around 7.5. I'm guessing they are going back and forth - equilibrium reached!
- With the 4 H + thing the atp just get ate up. But if you put it on both sides everything similar except for the atp/adp steady state values.

Theoretically, this should work forever. Unless ADP and Pi are sequestered by some other mechanisms, membrane proteins degrade to null, transport proteins get logged, incorrect species leave/enter through transport proteins, efflux not pumping out fast enough (could be rate-limiting)
___


In [7]:
#watermark
%reload_ext watermark
%watermark -v -p numpy,bokeh,colorcet,scipy,biocrnpyler,bioscrape,jupyterlab

CPython 3.7.7
IPython 7.13.0

numpy 1.18.1
bokeh 2.0.2
colorcet 2.0.2
scipy 1.4.1
biocrnpyler unknown
bioscrape 1.0.0
jupyterlab 1.2.6
